In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text
import rts.features.image

LOG = rts.utils.get_logger()

/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
sample_df = rts.metadata.get_one_percent_sample(df)
# # rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# # rts.metadata.export_metadata_stats(df, OUTDIR)
sample_df.head()

# medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)
# clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

# clip_df = rts.metadata.build_clips_df(LOCAL_VIDEOS, METADATA)
# rts.utils.dataframe_to_hdf5(METADATA, 'clips', clip_df)

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
mediaId,,,,,,,,,,,,,,,,,,,,
ZB001000,GE0111171458,/mnt/rts/0/0/0/ZB001000,1707,4:3,SD,2001-11-20T00:00:00Z,Programme,Téléjournal,"Téléjournal, flashes",None,TJ,TSR 1,Inconnu,LE 19:30 - 01.11.20,None,None,"[DUPLEX KABOUL :\r\nFAURE, Sébastien, envoyé s...",1670505500,1006214400,"{'794474': {'guid': 'GE0111171458', 'mediaId':..."
ZB002000,GE0212166889,/mnt/rts/0/0/0/ZB002000,530,4:3,SD,2002-12-30T00:00:00Z,Programme,Téléjournal,"Téléjournal, flashes",None,TJ,TSR 1,Inconnu,LE 12:45 - 02.12.30,None,None,[19:31.01\r\nGenève-ville : divers plans MAURY...,1670505550,1041206400,"{'1151573': {'guid': 'GE0212166889', 'mediaId'..."
ZB003000,GE0312152381,/mnt/rts/0/0/0/ZB003000,1677,4:3,SD,2003-12-30T00:00:00Z,Programme,Téléjournal,"Téléjournal, flashes",None,TJ,TSR 1,Inconnu,LE 19:30 - 03.12.30,None,None,"[TELEPHONE DE GENEVE SUR NEWS :\r\nGAILLARD, W...",1670505150,1072742400,"{'811444': {'guid': 'GE0312152381', 'mediaId':..."
ZB005000,GE0407175954,/mnt/rts/0/0/0/ZB005000,1897,4:3,SD,2004-07-18T00:00:00Z,Programme,Programme,Résultats et magazines sportifs,None,Dimanche Sport,TSR 2,Restriction/Condition,04.07.18,None,None,[00:03.26\nEntre Carcassonne et Nîmes dans l'A...,1670505299,1090108800,"{'1180751': {'guid': 'GE0407175954', 'mediaId'..."
ZB006000,GE0605146375,/mnt/rts/0/0/0/ZB006000,1541,4:3,SD,2006-05-27T00:00:00Z,Programme,Programme,Folklore et musique populaire,None,De si de la,TSR 1,Inconnu,L'HOMME QUI JOUE DU TOUR - 06.05.27,"* Bex : 20060527, et 200409, 200505, présentat...",[BEX],"[ Bex, diverses séquences : diverses présentat...",1670505351,1148688000,"{'837972': {'guid': 'GE0605146375', 'mediaId':..."


In [5]:
# sujets = rts.metadata.filter_by_asset_type(df, nested_struct='0/0')

In [9]:
# sujets

In [32]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = Path(OUTDIR) / 'test_trim.mp4'
# ok, outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)
# print(ok, outpath)
# outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)

ok, outpath = rts.io.media.trim(video_path,  ":", 300, 310)
print(ok, outpath)
if isinstance(outpath, io.BytesIO):
    print(len(outpath.getvalue()))

True <_io.BytesIO object at 0x7fdb889a61b0>
515797


In [7]:
rts.features.image.embed_images([Path(OUTDIR) / 'testori1.jpg', 
                                 Path(OUTDIR) / 'testori2.jpg'])

INFO:RTS:Load model: resnet50


array([[0.06276648, 1.1780145 , 0.3374293 , ..., 0.2396185 , 0.16062912,
        0.41618043],
       [0.28102228, 0.5253164 , 0.09940885, ..., 0.00276155, 0.019754  ,
        0.05477377]], dtype=float32)

In [18]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = Path(OUTDIR) / 'test_trim.mp4'
outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)


Args: ['-hide_banner', '-loglevel', 'error', '-i', '/media/data/rts/archive/0/0/0/ZB001000/ZB001000.mp4', '-filter_complex', '[0:v]trim=end=310:start=300[s0];[s0]setpts=PTS-STARTPTS[s1];[0:a]atrim=end=310:start=300[s2];[s2]asetpts=PTS-STARTPTS[s3];[s1][s3]concat=a=1:n=1:v=1[s4][s5]', '-map', '[s4]', '-map', '[s5]', '-movflags', 'faststart', 'data/test_trim.mp4', '-y']


In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [ ]:
medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)

In [ ]:
clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

In [ ]:
df = rts.metadata.build_clips_df(LOCAL_VIDEOS)

In [ ]:
atlases = rts.metadata.create_clip_texture_atlases(df, LOCAL_RTS_DATA,
                                                   '0', # folder name
                                                   tile_size=64,
                                                   flip=True,
                                                   no_border=True, 
                                                   format='jpg')

In [ ]:
import PIL
import os
from PIL import ImageOps, Image

In [ ]:
test_im_path = os.path.join(OUTDIR, 'test_image.jpg')
im = Image.open(test_im_path)
# rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

In [ ]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# audio_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000_audio.aac')
# rts.io.media.get_media_info(video_path)

# scenes = rts.io.media.detect_scenes(video_path)
# scenes = rts.io.media.filter_scenes(scenes, 5)
# clips = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)
# rts.pipeline.save_scenes(clips, OUTDIR)

In [5]:
test_video_folder = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000')
audio_path = os.path.join(test_video_folder, 'ZB001000_audio.m4a')

# outpath = rts.io.media.trim_media(video_path, 30, 40, Path(OUTDIR) / 'test_trim.mp4')
# rts.io.media.get_media_info(outpath)

In [ ]:
clips = rts.pipeline.extract_location_clips_from_transcript(transcript,
    25,
    OUTDIR,
    video_path,
    min_seconds=3,
    extend_duration=6,
    num_images=3,
    force=True
)

In [ ]:
imgs = list(rts.utils.recursive_glob(Path(OUTDIR) / 'images' / 'original', '.jpg'))
imgs

In [ ]:
rts.io.media.create_square_atlases(
    imgs,
    Path(OUTDIR) / 'atlas',
    width=512,
    max_tile_size=128,
    no_border=True,
    flip=True,
    atlas_prefix='small_atlas',
    bg_color=(255, 255, 255, 255)
)

In [ ]:
res = rts.features.text.enrich_scenes_with_transcript(clips.copy(), t)
# res

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)

rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=True, force_scene=False, force_trans=False)

In [ ]:
%ls {remux_folder}

In [ ]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

In [ ]:
fts

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))